In [11]:
import math

# --- 辅助函数 ---

def euclidean_distance(loc1, loc2):
    """计算两点间的欧几里得距离""" #这个要接如api
    return math.sqrt((loc1[0] - loc2[0]) ** 2 + (loc1[1] - loc2[1]) ** 2)

def check_route_feasibility(route, vehicle):
    """
    检查路线的时间窗和载重约束，并返回总行驶距离（作为成本指标）。
    route 为节点列表，要求 route[0] 与 route[-1]均为车辆起点（终点）。
    若不可行，则返回 None。
    """
    time = 0      # 假设车辆从起点出发的初始时间为 0，可以更改为vessel特有的起始时间
    load = 0
    total_distance = 0
    capacity = vehicle['capacity']
    current = route[0] #vessel 起始位置
    for nxt in route[1:]:
        travel = euclidean_distance(current["location"], nxt["location"]) #距离计算，需要调用api 函数，单位是时间
        total_distance += travel #需要乘以该段对应vessel的cost/h,计算总成本 
        arrival = time + travel #时间
        # 如果提前到达，则等待至时间窗开始
        start_service = max(arrival, nxt["tw"][0])
        if start_service > nxt["tw"][1]:
            return None
        load += nxt["demand"] #装载或者卸货
        if load > capacity or load < 0:
            return None
        time = start_service + nxt["service"]#装卸货时间
        current = nxt
    return total_distance 

def try_all_insertions(route, pickup_node, delivery_node, vehicle_id, vehicles):
    """
    在已有路线中枚举所有可能的位置插入 pickup_node 和 delivery_node
    （pickup 需插入在 delivery 之前），
    返回最优（增量成本最低）的方案：
         (最小增量成本, 新的路线)
    若没有可行方案，则返回 (None, None)。
    """
    best_cost = None
    best_route = None
    # 至少在起点之后插入 pickup，delivery 插入位置必须在 pickup 之后
    vehicle = vehicles[vehicle_id]
    for i in range(1, len(route)):
        for j in range(i + 1, len(route) + 1):
            new_route = route[:i] + [pickup_node] + route[i:j] + [delivery_node] + route[j:]
            new_cost = check_route_feasibility(new_route, vehicle)
            if new_cost is not None:
                # 原路线成本（新路线创建时原成本记作 0）
                old_cost = check_route_feasibility(route, vehicle) or 0
                incr = new_cost - old_cost
                if best_cost is None or incr < best_cost:
                    best_cost = incr
                    best_route = new_route
    return best_cost, best_route

# --- 简化后的调度算法 ---
def greedy_pdvrp_simple(requests, vehicles, rejection_penalty):
    """
    参数：
      requests: 每个请求为字典，包含：
           "id"      : 请求编号
           "pickup"  : 取货节点，字典格式 { "id": 节点编号,
                                               "type": "pickup",
                                               "location": (x, y),
                                               "tw": (开始时间, 结束时间),
                                               "service": 服务时间,
                                               "demand": 正数 }
           "delivery": 送货节点，字典格式 { "id": 节点编号,
                                               "type": "delivery",
                                               "location": (x, y),
                                               "tw": (开始时间, 结束时间),
                                               "service": 服务时间,
                                               "demand": 负数 }
      vehicles: 每辆车辆为字典，包含：
           "id"      : 车辆编号
           "start"   : 车辆起始节点（格式同 depot 节点，建议 service=0, demand=0）
           "capacity": 车辆最大载重
      rejection_penalty: 拒绝请求的成本惩罚（数值）

    算法思路：
      1. 对于每个请求（按输入顺序），尝试两种调度方式：
          a. 在已有车辆路线中查找最优的插入位置（pickup 与 delivery 均插入）。
          b. 从尚未用过的车辆中，新建一条路线，其形式为：
             [车辆起点, pickup, delivery, 车辆起点]。
      2. 选取两种方式中成本最小的方案，如果该成本小于拒绝惩罚，则采用该方案；
         否则直接拒绝该请求。
      3. 最后返回所有车辆的路线和被拒绝请求列表。
    """
    routes = []            # 已调度的车辆路线，每项格式为 {"vehicle": vehicle, "nodes": [...] }
    available_vehicles = vehicles.copy()  # 尚未使用的车辆
    rejected = []

    # 按请求顺序依次调度请求
    for req in requests:
        best_cost = None
        best_option = None  # ("existing", route_index, new_route) 或 ("new", vehicle, new_route)
        
        # 在已有路线中搜索插入方案
        for idx, route in enumerate(routes):
            cost, new_route = try_all_insertions(route["nodes"], req["pickup"], req["delivery"], route['vehicle']['id'], vehicles)
            if cost is not None:
                if best_cost is None or cost < best_cost:
                    best_cost = cost
                    best_option = ("existing", idx, new_route)
        
        # 在尚未使用的车辆中尝试新建路线
        for v in available_vehicles:
            new_route = [v["start"], req["pickup"], req["delivery"], v["start"]]
            cost_candidate = check_route_feasibility(new_route, v)
            if cost_candidate is not None:
                if best_cost is None or cost_candidate < best_cost:
                    best_cost = cost_candidate
                    best_option = ("new", v, new_route)
        
        # 如果找到的最优方案成本低于拒绝惩罚，则采用该方案；否则拒绝该请求
        if best_option is None or best_cost >= rejection_penalty*req['pickup']['demand']:
            rejected.append(req)
        # if best_cost >= rejection_penalty*req['pickup']['demand']:
        #     rejected.append(req)
        else:
            if best_option[0] == "existing":
                # 更新对应车辆路线
                idx, new_route = best_option[1], best_option[2]
                routes[idx]["nodes"] = new_route
            elif best_option[0] == "new":
                vehicle, new_route = best_option[1], best_option[2]
                routes.append({"vehicle": vehicle, "nodes": new_route})
                available_vehicles.remove(vehicle)
    
    return routes, rejected

# --- 示例数据及测试 ---
def main():
    # 定义车辆，车辆起始节点作为各自的出发点（同时作为终点），载重不同
    vehicle0 = {
        "id": 0,
        "start": {"id": "veh1_start", "type": "depot", "location": (50, 50), "tw": (0, 1000), "service": 0, "demand": 0},
        "capacity": 15
    }
    vehicle1 = {
        "id": 1,
        "start": {"id": "veh2_start", "type": "depot", "location": (10, 10), "tw": (0, 1000), "service": 0, "demand": 0},
        "capacity": 10
    }
    vehicle2 = {
        "id": 2,
        "start": {"id": "veh3_start", "type": "depot", "location": (80, 80), "tw": (0, 1000), "service": 0, "demand": 0},
        "capacity": 12
    }
    vehicles = [vehicle0, vehicle1, vehicle2]
    
    # 定义请求（pickup 与 delivery 节点示例数据）
    req1 = {
        "id": 1,
        "pickup": {"id": "1p", "type": "pickup", "location": (20, 30), "tw": (10, 100), "service": 10, "demand": 5},
        "delivery": {"id": "1d", "type": "delivery", "location": (60, 20), "tw": (50, 150), "service": 10, "demand": -5}
    }
    req2 = {
        "id": 2,
        "pickup": {"id": "2p", "type": "pickup", "location": (30, 40), "tw": (20, 120), "service": 10, "demand": 7},
        "delivery": {"id": "2d", "type": "delivery", "location": (80, 30), "tw": (60, 180), "service": 10, "demand": -7}
    }
    req3 = {
        "id": 3,
        "pickup": {"id": "3p", "type": "pickup", "location": (25, 60), "tw": (15, 110), "service": 10, "demand": 6},
        "delivery": {"id": "3d", "type": "delivery", "location": (70, 70), "tw": (70, 200), "service": 10, "demand": -6}
    }
    req4 = {
        "id": 4,
        "pickup": {"id": "4p", "type": "pickup", "location": (55, 60), "tw": (30, 130), "service": 10, "demand": 4},
        "delivery": {"id": "4d", "type": "delivery", "location": (90, 90), "tw": (80, 220), "service": 10, "demand": -4}
    }
    requests = [req1, req2, req3, req4]
    
    # 拒绝惩罚成本
    rejection_penalty = 200
    
    routes, rejected = greedy_pdvrp_simple(requests, vehicles, rejection_penalty)
    
    # 输出最终解
    print("最终调度方案：")
    print(routes)


if __name__ == "__main__":
    main()


最终调度方案：
[{'vehicle': {'id': 0, 'start': {'id': 'veh1_start', 'type': 'depot', 'location': (50, 50), 'tw': (0, 1000), 'service': 0, 'demand': 0}, 'capacity': 15}, 'nodes': [{'id': 'veh1_start', 'type': 'depot', 'location': (50, 50), 'tw': (0, 1000), 'service': 0, 'demand': 0}, {'id': '2p', 'type': 'pickup', 'location': (30, 40), 'tw': (20, 120), 'service': 10, 'demand': 7}, {'id': '1p', 'type': 'pickup', 'location': (20, 30), 'tw': (10, 100), 'service': 10, 'demand': 5}, {'id': '1d', 'type': 'delivery', 'location': (60, 20), 'tw': (50, 150), 'service': 10, 'demand': -5}, {'id': '2d', 'type': 'delivery', 'location': (80, 30), 'tw': (60, 180), 'service': 10, 'demand': -7}, {'id': 'veh1_start', 'type': 'depot', 'location': (50, 50), 'tw': (0, 1000), 'service': 0, 'demand': 0}]}, {'vehicle': {'id': 2, 'start': {'id': 'veh3_start', 'type': 'depot', 'location': (80, 80), 'tw': (0, 1000), 'service': 0, 'demand': 0}, 'capacity': 12}, 'nodes': [{'id': 'veh3_start', 'type': 'depot', 'location': (